##  EDA (Reviews de States de Google)

In [15]:
# Importamos las bibliotecas necesarias
import findspark
findspark.init()
import pyspark
findspark.find()
import os

In [16]:
# Importamos las bibliotecas necesarias para Spark y definimos alias
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime

# Configuramos Spark para poder procesar de forma local archivos de gran tamaño
conf = SparkConf().setAppName('appName').setMaster('local') \
    .set("spark.driver.memory", "12g") \
    .set("spark.executor.memory", "10g") \
    .set("spark.executor.cores", "4") \
    .set("spark.dynamicAllocation.maxExecutors", "2")


sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

Establecemos una sesión de Spark para gestionar el procesamiento

In [17]:
# Inicializamos la sesión de Spark
spark

In [4]:
# Importamos la biblioteca para Koalas
import collections.abc
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping
collections.Callable = collections.abc.Callable

import databricks.koalas as ks

Hemos creado una función que nos permite extraer y concatenar los datos de los cinco estados seleccionados. Dado el volumen considerable de datos con el que estamos tratando, hemos tomado la decisión de aprovechar las ventajas de las herramientas Spark y Koalas para lograr un procesamiento más eficiente.

In [5]:
# Se define una función para leer archivos Parquet y convertirlos a DataFrames de Koalas
def read_parquet_files(directory, estado):
    dataframes_koalas = []

    for archivo in os.listdir(directory):
        if archivo.endswith('.parquet'):
            ruta_archivo = os.path.join(directory, archivo)
            df_spark = spark.read.parquet(ruta_archivo)
            df_koalas = ks.DataFrame(df_spark)  # Convertimos a DataFrame de Koalas
            df_koalas['state'] = estado  # Agregamos la columna "Estado"
            dataframes_koalas.append(df_koalas)

    return ks.concat(dataframes_koalas, ignore_index=True)

# Directorios de los estados
directorio_California = r'C:\Escritorio\PF\states_reviews\states_reviews\review-California'
directorio_Florida = r'C:\Escritorio\PF\states_reviews\states_reviews\review-Florida'
directorio_Illinois = r'C:\Escritorio\PF\states_reviews\states_reviews\review-Illinois'
directorio_New_York = r'C:\Escritorio\PF\states_reviews\states_reviews\review-New_York'
directorio_Texas = r'C:\Escritorio\PF\states_reviews\states_reviews\review-Texas'

# Leer y concatenar los DataFrames para cada estado
df_California = read_parquet_files(directorio_California, 'California')
df_Florida = read_parquet_files(directorio_Florida, 'Florida')
df_Illinois = read_parquet_files(directorio_Illinois, 'Illinois')
df_New_York = read_parquet_files(directorio_New_York, 'New York')
df_Texas = read_parquet_files(directorio_Texas, 'Texas')

In [6]:
# Concatenamos los DataFrames de los estados en uno solo
df_reviews_top_5 = ks.concat([df_California, df_Florida, df_Illinois, df_New_York, df_Texas])
df_reviews_top_5.head()

,user_id,name,time,rating,text,pics,resp,gmap_id,state
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
2,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
3,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
4,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California


In [7]:
# Obtenemos la longitud del DataFrame
len(df_reviews_top_5)

11746824

In [8]:
# Controlamos valores nulos en el DataFrame
df_reviews_top_5.isnull().sum()

user_id           0
name              0
time              0
rating            0
text        4870400
pics       11348752
resp       10246105
gmap_id           0
state             0
Name: 0, dtype: int64

Dada la cantidad de valores nulos con los que contamos en las columnas 'text', 'pics' y 'resp', consideramos que la mejor decisión es eliminar los mismos de la base de datos

In [9]:
# Creamos un nuevo DataFrame eliminando columnas específicas
df_reviews_top_5_clean = df_reviews_top_5.drop(columns=['text', 'pics', 'resp'])

In [10]:
# Mostramos las primeras filas del DataFrame limpio
df_reviews_top_5_clean.head()

,user_id,name,time,rating,gmap_id,state
0,108991152262655788985,Song Ro,1609909927056,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
1,111290322219796215751,Rafa Robles,1612849648663,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
2,112640357449611959087,David Han,1583643882296,4,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
3,117440349723823658676,Anthony Kim,1551938216355,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California
4,100580770836123539210,Mario Marzouk,1494910901933,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California


Eliminamos las filas con valores duplicados para evitar que afecten en nuestro análisis

In [11]:
# Eliminamos las filas duplicadas en el DataFrame
df_reviews_google_top_5 = df_reviews_top_5_clean.drop_duplicates()
df_reviews_google_top_5.head()

,user_id,name,time,rating,gmap_id,state
1812280,100000019059943251257,Jim Hajek,1532172902488,5,0x880fd36b093a9a07:0x940cc06f90294db,Illinois
1418443,100000019059943251257,Jim Hajek,1542748194590,5,0x880e4cae16c03f93:0x8ad15acaf8ffd496,Illinois
466029,100000020623254171349,Zachary Kular,1498657367260,4,0x880fab1b8261c301:0xc13854f55b0feec4,Illinois
1792773,100000020623254171349,Zachary Kular,1558493360838,5,0x880e2de4633757b5:0x4679b7845e3ee752,Illinois
1735594,100000029979508209337,Raychel Perez,1593401857411,5,0x80eacb93b18677bb:0x6eb86c000772bf33,California


Obtenemos el total de filas del DataFrame resultante para comprender con cuantos datos contamos de reviews de los cinco estados correspondientes

In [16]:
# Obtenemos la longitud del nuevo DataFrame sin duplicados
len(df_reviews_google_top_5)

11392427

Verificamos la presencia de valores atípicos en la columna 'rating'

In [17]:
# Controlamos que en el rating no haya valores outliers
unique_ratings = df_reviews_google_top_5["rating"].unique().to_list()
print(unique_ratings)

[5, 1, 3, 2, 4]


In [12]:
# Volvemos a convertir el DataFrame de Koala a Spark
df_reviews_google_top_5 = df_reviews_google_top_5.to_spark()

# Mostrar el DataFrame Spark resultante
df_reviews_google_top_5.show()

+--------------------+--------------------+-------------+------+--------------------+----------+
|             user_id|                name|         time|rating|             gmap_id|     state|
+--------------------+--------------------+-------------+------+--------------------+----------+
|10000001905994325...|           Jim Hajek|1532172902488|     5|0x880fd36b093a9a0...|  Illinois|
|10000001905994325...|           Jim Hajek|1542748194590|     5|0x880e4cae16c03f9...|  Illinois|
|10000002062325417...|       Zachary Kular|1498657367260|     4|0x880fab1b8261c30...|  Illinois|
|10000002062325417...|       Zachary Kular|1558493360838|     5|0x880e2de4633757b...|  Illinois|
|10000002997950820...|       Raychel Perez|1593401857411|     5|0x80eacb93b18677b...|California|
|10000004867909222...|        Ruby Johnson|1607728724073|     4|0x864db2282e13047...|     Texas|
|10000008541895975...|      Kevin Broadway|1545840301933|     5|0x8893892abc9823b...|   Florida|
|10000011422746342...|      st

Obtenemos la suma y porcentaje final de los valores de la columna 'rating' para entender la distribución y puntuación general de los usuarios

In [19]:
total_rows = df_reviews_google_top_5.count()
# Calcular la cantidad de ocurrencias para cada valor de 'rating'
rating_counts = df_reviews_google_top_5.groupBy('rating').count()
# Calcular el porcentaje de cada valor de 'rating'
rating_percentages = rating_counts.withColumn('percentage', (col('count') / total_rows) * 100)
rating_percentages.show()

+------+-------+-----------------+
|rating|  count|       percentage|
+------+-------+-----------------+
|     5|7274308| 63.8521361602756|
|     1| 722411|6.341151011983663|
|     3| 951471|8.351784918174152|
|     2| 348219|3.056583114379403|
|     4|2096018|18.39834479518719|
+------+-------+-----------------+



Transformamos la columna 'time' de su formato original para poder obtener las fechas en un formato legible

In [13]:
# Convertimos milisegundos a segundos y aplicamos el formato de tiempo
df_reviews_google_top_5 = df_reviews_google_top_5.withColumn("time", from_unixtime(df_reviews_google_top_5["time"]/1000))
# Mostrar el DataFrame resultante
df_reviews_google_top_5.show()

+--------------------+--------------------+-------------------+------+--------------------+----------+
|             user_id|                name|               time|rating|             gmap_id|     state|
+--------------------+--------------------+-------------------+------+--------------------+----------+
|10000001905994325...|           Jim Hajek|2018-07-21 13:35:02|     5|0x880fd36b093a9a0...|  Illinois|
|10000001905994325...|           Jim Hajek|2018-11-20 22:09:54|     5|0x880e4cae16c03f9...|  Illinois|
|10000002062325417...|       Zachary Kular|2017-06-28 15:42:47|     4|0x880fab1b8261c30...|  Illinois|
|10000002062325417...|       Zachary Kular|2019-05-22 04:49:20|     5|0x880e2de4633757b...|  Illinois|
|10000002997950820...|       Raychel Perez|2020-06-29 05:37:37|     5|0x80eacb93b18677b...|California|
|10000004867909222...|        Ruby Johnson|2020-12-12 00:18:44|     4|0x864db2282e13047...|     Texas|
|10000008541895975...|      Kevin Broadway|2018-12-26 17:05:01|     5|0x8

Y por último importamos el DataFrame resultante para poder exportarlo a la base de datos

In [ ]:
df_reviews_google_top_5 = df_reviews_google_top_5.coalesce(1)

# Ruta para guardar el archivo Parquet
ruta_exportacion = 'C:/Escritorio/PF/states_reviews/reviews_google_top_5.parquet'

# Guardar el DataFrame en formato Parquet con una sola partición
df_reviews_google_top_5.write.parquet(ruta_exportacion, compression='snappy')

In [2]:
import pandas as pd

In [12]:
df1 = pd.read_parquet(r'C:\Escritorio\PF\states_reviews\reviews_google_top_5.parquet')

In [18]:
df_spark = spark.read.parquet(r'C:\Escritorio\PF\states_reviews\reviews_google_top_5.parquet')

In [19]:
df_spark.show()

+--------------------+--------------------+-------------------+------+--------------------+----------+
|             user_id|                name|               time|rating|             gmap_id|     state|
+--------------------+--------------------+-------------------+------+--------------------+----------+
|10000001905994325...|           Jim Hajek|2018-07-21 13:35:02|     5|0x880fd36b093a9a0...|  Illinois|
|10000001905994325...|           Jim Hajek|2018-11-20 22:09:54|     5|0x880e4cae16c03f9...|  Illinois|
|10000002062325417...|       Zachary Kular|2017-06-28 15:42:47|     4|0x880fab1b8261c30...|  Illinois|
|10000002062325417...|       Zachary Kular|2019-05-22 04:49:20|     5|0x880e2de4633757b...|  Illinois|
|10000002997950820...|       Raychel Perez|2020-06-29 05:37:37|     5|0x80eacb93b18677b...|California|
|10000004867909222...|        Ruby Johnson|2020-12-12 00:18:44|     4|0x864db2282e13047...|     Texas|
|10000008541895975...|      Kevin Broadway|2018-12-26 17:05:01|     5|0x8

In [13]:
df1

,user_id,name,time,rating,gmap_id,state
0,100000019059943251257,Jim Hajek,2018-07-21 13:35:02,5,0x880fd36b093a9a07:0x940cc06f90294db,Illinois
1,100000019059943251257,Jim Hajek,2018-11-20 22:09:54,5,0x880e4cae16c03f93:0x8ad15acaf8ffd496,Illinois
2,100000020623254171349,Zachary Kular,2017-06-28 15:42:47,4,0x880fab1b8261c301:0xc13854f55b0feec4,Illinois
3,100000020623254171349,Zachary Kular,2019-05-22 04:49:20,5,0x880e2de4633757b5:0x4679b7845e3ee752,Illinois
4,100000029979508209337,Raychel Perez,2020-06-29 05:37:37,5,0x80eacb93b18677bb:0x6eb86c000772bf33,California
...,...,...,...,...,...,...
11392422,118446479271677041364,Steve Ramirez,2019-07-10 05:06:15,3,0x4065fd3cb737c7ef:0x9f82f8da541e57c8,New York
11392423,118446545917461448334,Kristin Miliman,2020-05-19 18:09:39,5,0x88e77f21c6b80e17:0x4c31943c871d58e8,Florida
11392424,118446589693008870662,Austen Rowell,2019-07-30 01:10:42,5,0x8640cddd118a84c5:0x84574508cf1a6b67,Texas
11392425,118446649441296359833,Alec Holden,2020-03-16 22:09:25,5,0x80dd2de9b5a1f831:0x66198da60590b5a0,California


In [14]:
df_reviews_google_top_5.dtypes

[('user_id', 'string'),
 ('name', 'string'),
 ('time', 'string'),
 ('rating', 'bigint'),
 ('gmap_id', 'string'),
 ('state', 'string')]

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet(r'C:\Escritorio\PF\states_reviews\reviews_google_top_5.parquet')

In [3]:
import os
import mysql.connector
password = os.environ.get('PASSWORD')

In [4]:
connection = mysql.connector.connect(host='databasegy.cdmolmugarf8.us-west-1.rds.amazonaws.com', 
                                     port='3306', 
                                     user='data13', 
                                     password = password, 
                                     database='database13')
cursor = connection.cursor()

In [5]:
connection.start_transaction()

In [6]:
from tqdm import tqdm

In [9]:
total_rows = len(df)
progress_bar = tqdm(total=total_rows, desc="Inserting Data")

Inserting Data:   0%|          | 0/11392427 [00:00<?, ?it/s]

In [10]:
# Nombre de la tabla que deseas crear
table_name = "Reviews_Google"

In [8]:
# Define la estructura de la tabla (columnas y tipos de datos)
create_table = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    user_id VARCHAR(50),
    name VARCHAR(50),
    time VARCHAR(50),
    rating INT,
    gmap_id VARCHAR(50),
    state VARCHAR(50)
) ENGINE INNODB
"""

In [ ]:
# Ejecuta la consulta para crear la tabla
cursor.execute(create_table)

In [ ]:
# Inserta los datos del DataFrame en la tabla
for _, row in df.iterrows():
    insert_query = f"""
    INSERT INTO {table_name} (user_id, name, time, rating, gmap_id, state)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        row['user_id'],
        row['name'],
        row['time'],
        row['rating'],
        row['gmap_id'],
        row['state']
    ))

    # Actualiza el progreso en el tqdm
    progress_bar.update(1)